In [12]:
link = 'https://www.imdb.com/search/title/?user_rating=2,4&num_votes=4000,&sort=user_rating,asc'
title_link_class = 'ipc-title-link-wrapper'
reviews_link_class = 'isReview'
text_div = 'ipc-html-content-inner-div'
spoiler = 'Expand Spoiler' #aria-label
more_btn = 'ipc-btn ipc-btn--single-padding ipc-btn--center-align-content ipc-btn--default-height ipc-btn--core-base ipc-btn--theme-base ipc-btn--button-radius ipc-btn--on-accent2 ipc-text-button ipc-see-more__button'
stars = 'ipc-rating-star--rating'

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Setup the WebDriver
driver = webdriver.Chrome()  
driver.get(link)
wait = WebDriverWait(driver, 1)

# Step 1: Load more movies
for _ in range(5):  # Load more movies 5 times
    try:
        more_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, more_btn)))
        more_btn.click()
        time.sleep(1)  # Give time for new movies to load
    except Exception as e:
        print("Error loading more movies:", e)
        break


In [14]:

# Step 2: Collect movie links
movies = driver.find_elements(By.CLASS_NAME, title_link_class)
movie_links = [movie.get_attribute('href') for movie in movies]
print(movie_links)

['https://www.imdb.com/title/tt0804492/?ref_=sr_t_1', 'https://www.imdb.com/title/tt19857862/?ref_=sr_t_2', 'https://www.imdb.com/title/tt7668842/?ref_=sr_t_3', 'https://www.imdb.com/title/tt5290620/?ref_=sr_t_4', 'https://www.imdb.com/title/tt0343788/?ref_=sr_t_5', 'https://www.imdb.com/title/tt0317676/?ref_=sr_t_6', 'https://www.imdb.com/title/tt1595842/?ref_=sr_t_7', 'https://www.imdb.com/title/tt0367677/?ref_=sr_t_8', 'https://www.imdb.com/title/tt0096149/?ref_=sr_t_9', 'https://www.imdb.com/title/tt0086026/?ref_=sr_t_10', 'https://www.imdb.com/title/tt0844029/?ref_=sr_t_11', 'https://www.imdb.com/title/tt2814362/?ref_=sr_t_12', 'https://www.imdb.com/title/tt1183733/?ref_=sr_t_13', 'https://www.imdb.com/title/tt0131550/?ref_=sr_t_14', 'https://www.imdb.com/title/tt0174917/?ref_=sr_t_15', 'https://www.imdb.com/title/tt0469999/?ref_=sr_t_16', 'https://www.imdb.com/title/tt16228146/?ref_=sr_t_17', 'https://www.imdb.com/title/tt7255568/?ref_=sr_t_18', 'https://www.imdb.com/title/tt0089

In [15]:
len(movie_links)


406

In [ ]:


# Initialize lists to store data
titles = []
reviews = []
ratings = []

# Visit each moviexecute_scripte page and collect reviews
for movie_link in movie_links:
    driver.get(movie_link)
    time.sleep(1)  # Wait for movie page to load

    # Navigate to the reviews page
    try:
        # driver.("window.scrollTo(0, document.body.scrollHeight * 0.2);")
        review_link = wait.until(EC.presence_of_element_located((By.XPATH, "//a[contains(@href, '/reviews/') and contains(@class, 'isReview')]")))
        review_link.click()
        time.sleep(1)

        # Locate all review cards
        review_cards = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="review-card-parent"]')

        # Collect up to 20 reviews
        for i, review_card in enumerate(review_cards[:40]):
            try:
                # Get the star rating if available
                try:
                    rating = review_card.find_element(By.CLASS_NAME, 'ipc-rating-star--rating').text
                except:
                    rating = 'No rating'  # If no rating is present

                # Expand spoiler if the button is present
                try:
                    spoiler_button = review_card.find_element(By.CLASS_NAME, 'review-spoiler-button')
                    spoiler_button.click()
                    time.sleep(0.2)  # Allow time for text to load
                except:
                    pass  # If no spoiler button is present, continue

                # Extract the full review text
                review_text = review_card.find_element(By.CLASS_NAME, 'ipc-html-content-inner-div').text

                # Save data
                titles.append(driver.title.split(" - ")[0])  # Movie title
                reviews.append(review_text)
                ratings.append(rating)
                print(i, ": ", review_text[:10])

            except Exception as e:
                print(f"Error processing review {i+1} on {movie_link}: {e}")
                continue

    except Exception as e:
        print(f"Couldn't find reviews link for {movie_link}: {e}")
        continue

# Save data to Excel
data = {
    'Title': titles,
    'Review': reviews,
    'Stars': ratings
}
df = pd.DataFrame(data)
df.to_excel('imdb_reviews_negative_2_4.xlsx', index=False)

# Close the WebDriver
driver.quit()


In [8]:
ata = {
    'Title': titles,
    'Review': reviews,
    'Stars': ratings
}
df = pd.DataFrame(data)
df.to_excel('imdb_reviews_3.xlsx', index=False)

# Close the WebDriver
driver.quit()

In [ ]:
import pandas as pd
from deep_translator import GoogleTranslator

# Load the Excel file
file_path = 'imdb_reviews.xlsx'  # Path to your Excel file
df = pd.read_excel(file_path)

# Initialize the translator
translator = GoogleTranslator(source='en', target='uk')

# Translate each review and title
df['Title'] = df['Title'].apply(lambda x: translator.translate(x) if pd.notnull(x) else x)
df['Review'] = df['Review'].apply(lambda x: translator.translate(x) if pd.notnull(x) else x)

# Save the translated file
translated_file_path = 'imdb_reviews_ukrainian.xlsx'
df.to_excel(translated_file_path, index=False)

print(f"Translation completed and saved to {translated_file_path}")
